<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#外匯保證金-&amp;-損益-算法" data-toc-modified-id="外匯保證金-&amp;-損益-算法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>外匯保證金 &amp; 損益 算法</a></span></li><li><span><a href="#1.-累積獲利為正時平倉，並買進0.01手；若為負，則持續加碼(變成2倍手)買進" data-toc-modified-id="1.-累積獲利為正時平倉，並買進0.01手；若為負，則持續加碼(變成2倍手)買進-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. 累積獲利為正時平倉，並買進0.01手；若為負，則持續加碼(變成2倍手)買進</a></span></li><li><span><a href="#2.-累積獲利為正時平倉，並買進0.01手；若為負，則在符合條件(限制最大持有手數)下持續加碼買進" data-toc-modified-id="2.-累積獲利為正時平倉，並買進0.01手；若為負，則在符合條件(限制最大持有手數)下持續加碼買進-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. 累積獲利為正時平倉，並買進0.01手；若為負，則在符合條件(限制最大持有手數)下持續加碼買進</a></span></li></ul></div>

In [79]:
#基本功能
import numpy as np
import pandas as pd

#繪圖
import plotly.graph_objects as go
from plotly import subplots
import plotly.express as px

#TEJ
import tejapi
tejapi.ApiConfig.api_key = "Your Key"
tejapi.ApiConfig.ignoretz = True

In [80]:
fx = tejapi.get('GLOBAL/GCURR',
                coid = 'EUR',
                mdate={'gte': '2019-01-01', 'lte':'2020-12-31'},
                opts = {'columns':['mdate','tuse2']},
                chinese_column_name = True,
                paginate = True)

In [81]:
#歐元兌美元較常見, 報價頂多到第五位 (步)
fx['原幣兌美元 (美元)'] = fx['原幣兌美元 (美元)'].apply(lambda x: round(x,5))
fx

,日期,原幣兌美元 (美元)
None,,
0,2019-01-01,1.14653
1,2019-01-02,1.13430
2,2019-01-03,1.13947
3,2019-01-04,1.13973
4,2019-01-07,1.14758
...,...,...
517,2020-12-24,1.21862
518,2020-12-28,1.22145
519,2020-12-29,1.22504


In [82]:
fig = px.line(fx, x="日期", y="原幣兌美元 (美元)")
fig.show()

# 外匯保證金 & 損益 算法
- 一手:100,000歐元  (如果槓桿比例1:500, 保證金:200歐元) > 1步賺 1美                
- 0.01手: 1000歐元  (如果槓桿比例1:500, 保證金:2歐元)   > 1步賺 0.01美

- 參考網站: https://abbymt4mt5.com/2021/05/29/forex-learning-1-3/
https://abbymt4mt5.com/2019/04/11/badtrade-01/ 

# 1. 累積獲利為正時平倉，並買進0.01手；若為負，則持續加碼(變成2倍手)買進

In [83]:
account  = 100000       # 10萬美金
lot = 0.01              # 0.01手
lot_record = []         # 倉位紀錄
lev = 1/500             # 槓桿比例
account_record = []   
cum_profit = 0

In [84]:
for i in range(len(fx)):
    
    #第一筆買入
    if i < 1:
        cum_profit -= (100000*lot*lev*fx.loc[i, '原幣兌美元 (美元)']) #初始買0.01手需要的保證金為歐元，所以按照當時匯率換成美元
        account += cum_profit                                       
        account_record.append(account)
        lot_record.append(lot)
        
        
    else:
        #累計損益
        cum_profit += (fx.loc[i, '原幣兌美元 (美元)'] - fx.loc[i-1, '原幣兌美元 (美元)'])*100000*lot #乘以100000是為了換成布數
        #帳戶餘額
        account += (fx.loc[i, '原幣兌美元 (美元)'] - fx.loc[i-1, '原幣兌美元 (美元)'])*100000*lot
        
        #如果累計損益獲利,平倉後買入0.01手
        if cum_profit >= 0:       
            #買入0.01手，故重置lot & cum_profit
            lot = 0.01
            cum_profit = -(100000*lot*lev*fx.loc[i, '原幣兌美元 (美元)']) 
            account += cum_profit
            account_record.append(account)
            lot_record.append(lot)
        #若累計損益為負，則加碼買進
        else:
            lot = lot *2 
            cum_profit -= (100000*(lot-lot/2)*lev*fx.loc[i, '原幣兌美元 (美元)'])  #lot-lot/2為新增購買手數，一樣要換成美元
            account -= (100000*(lot-lot/2)*lev*fx.loc[i, '原幣兌美元 (美元)'])
            account_record.append(account)              
            lot_record.append(lot)    

In [85]:
# 通常累計損益轉為正時，帳戶餘額暴增
account_record

[99997.70694,
 99983.20834000001,
 99988.99046000002,
 99980.91262,
 100041.41746000001,
 100035.70914,
 100054.0202,
 100047.2003,
 100036.29314,
 100028.67569999999,
 99964.33297999999,
 99892.51826,
 99819.60882,
 99491.75474,
 99234.06001999999,
 98521.81169999999,
 98344.61873999998,
 88523.00466000014,
 110188.56207999976,
 110187.84635999976,
 110184.31283999975,
 110200.93661999975,
 110195.35697999975,
 110192.87349999975,
 110174.80437999975,
 110131.51621999976,
 110028.75269999976,
 109881.93413999976,
 109638.41413999976,
 108745.54245999976,
 109407.23589999977,
 104987.39589999974,
 106196.35077999986,
 101569.14053999986,
 98037.65125999923,
 124331.70266000052,
 124329.05482000051,
 124324.00018000051,
 124313.89298000051,
 124315.31938000052,
 124362.64148000052,
 124358.42746000052,
 124354.13890000051,
 124343.48490000052,
 124302.70218000052,
 124219.31114000051,
 124142.77738000051,
 123262.22442000051,
 123522.42282000049,
 123171.96906000048,
 125381.55146000054

In [86]:
# 購買過多手 >>> 應該建立停損機制/調控風險機制
lot_record

[0.01,
 0.02,
 0.04,
 0.08,
 0.01,
 0.02,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 0.01,
 0.02,
 0.04,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 40.96,
 81.92,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 40.96,
 0.01,
 0.02,
 0.04,
 0.08,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.01,
 0.01,
 0.02,
 0.04,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 0.01,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,


In [87]:
fx['價值'] = account_record

In [88]:
fig = px.line(fx, x="日期", y="價值")
fig.show()

# 2. 累積獲利為正時平倉，並買進0.01手；若為負，則在符合條件(限制最大持有手數)下持續加碼買進

In [72]:
fx = tejapi.get('GLOBAL/GCURR',
                coid = 'EUR',
                mdate={'gte': '2019-01-01', 'lte':'2020-12-31'},
                opts = {'columns':['mdate','tuse2']},
                chinese_column_name = True,
                paginate = True)
fx['原幣兌美元 (美元)'] = fx['原幣兌美元 (美元)'].apply(lambda x: round(x,5))

In [73]:
account  = 100000            
lot = 0.01                   
lot_record = []
lev = 1/500               
account_record = []   
cum_profit = 0

In [74]:
for i in range(len(fx)):
    
    #第一筆買入
    if i < 1:
        cum_profit -= (100000*lot*lev*fx.loc[i, '原幣兌美元 (美元)'])
        account += cum_profit                                       
        account_record.append(account)
        lot_record.append(lot)
        
    else:
        #累計損益
        cum_profit += (fx.loc[i, '原幣兌美元 (美元)'] - fx.loc[i-1, '原幣兌美元 (美元)'])*100000*lot #呈上100000是為了換成步數
        #帳戶餘額
        account += (fx.loc[i, '原幣兌美元 (美元)'] - fx.loc[i-1, '原幣兌美元 (美元)'])*100000*lot
        
        #如果累計損益獲利,平倉後買入0.01手
        if cum_profit >= 0:       
            #買入0.01手，故重置lot & cum_profit
            lot = 0.01
            cum_profit = -(100000*lot*lev*fx.loc[i, '原幣兌美元 (美元)']) 
            account += cum_profit
            account_record.append(account)
            lot_record.append(lot)
        #若累計損益為負，則加碼買進
        else:
            if lot_record[-1] < 20:    #當持有手小於20，才加碼
                lot = lot *2 
                cum_profit -= (100000*(lot-lot/2)*lev*fx.loc[i, '原幣兌美元 (美元)'])
                account -= (100000*(lot-lot/2)*lev*fx.loc[i, '原幣兌美元 (美元)'])
                account_record.append(account)              
                lot_record.append(lot)    
            else:                      #若大於20，仍然損失，則實現損失，並購買0.01手
            #停損(實現損失)，並買入0.01手: 故重置lot & cum_profit
                lot = 0.01
                cum_profit = -(100000*lot*lev*fx.loc[i, '原幣兌美元 (美元)']) 
                account += cum_profit
                account_record.append(account)
                lot_record.append(lot)

In [75]:
account_record

[99997.70694,
 99983.20834000001,
 99988.99046000002,
 99980.91262,
 100041.41746000001,
 100035.70914,
 100054.0202,
 100047.2003,
 100036.29314,
 100028.67569999999,
 99964.33297999999,
 99892.51826,
 99819.60882,
 99491.75474,
 99234.06001999999,
 98521.81169999999,
 98344.61873999998,
 88523.00466000014,
 110188.56207999976,
 110187.84635999976,
 110184.31283999975,
 110200.93661999975,
 110195.35697999975,
 110192.87349999975,
 110174.80437999975,
 110131.51621999976,
 110028.75269999976,
 109881.93413999976,
 109638.41413999976,
 108745.54245999976,
 109407.23589999977,
 104987.39589999974,
 106196.35077999986,
 106194.09139999986,
 106193.22921999985,
 106197.38061999985,
 106194.73277999985,
 106189.67813999984,
 106179.57093999984,
 106180.99733999986,
 106228.31943999985,
 106224.10541999985,
 106219.81685999985,
 106209.16285999985,
 106168.38013999985,
 106084.98909999985,
 106008.45533999984,
 105127.90237999984,
 105388.10077999982,
 105037.64701999981,
 107247.2294199998

In [76]:
lot_record

[0.01,
 0.02,
 0.04,
 0.08,
 0.01,
 0.02,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 0.01,
 0.02,
 0.04,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 0.01,
 0.02,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.01,
 0.01,
 0.02,
 0.04,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 0.01,
 0.01,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.32,
 0.64,
 1.28,
 2.56,
 5.12,
 10.24,
 20.48,
 0.01,
 0.02,
 0.04,
 0.08,
 0.16,
 0.

In [77]:
fx['價值'] = account_record

In [78]:
fig = px.line(fx, x="日期", y="價值")
fig.show()